In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv',low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,x,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [9]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model = RandomForestRegressor(n_estimators=100,max_depth=5,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [11]:
df.head

<bound method NDFrame.head of        Unnamed: 0                    id asins  ... websiteids weight brand_cat
0               0  AVpfHrJ6ilAPnD_xVXOI   NaN  ...        NaN    NaN         0
1               1  AVpfHrJ6ilAPnD_xVXOI   NaN  ...        NaN    NaN         0
2               2  AVpfHsWP1cnluZ0-eVZ7   NaN  ...        NaN    NaN         1
3               3  AVpfHsWP1cnluZ0-eVZ7   NaN  ...        NaN    NaN         1
4               4  AVpfHsWP1cnluZ0-eVZ7   NaN  ...        NaN    NaN         1
...           ...                   ...   ...  ...        ...    ...       ...
18275       19382  AVpfdSjlilAPnD_xcGPm   NaN  ...        NaN    NaN      1731
18276       19383  AVpf3bFWilAPnD_xjrQ2   NaN  ...        NaN    NaN       299
18277       19384  AVpf0fJXLJeJML43EVe9   NaN  ...        NaN    NaN       399
18278       19385  AVpf0fJXLJeJML43EVe9   NaN  ...        NaN    NaN       399
18279       19386  AVpf0fJXLJeJML43EVe9   NaN  ...        NaN    NaN       399

[18280 rows x 50 colu

In [14]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [15]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"','"'))

df['features_parsed'] = df['features'].map(parse_features)


In [0]:
df['features_parsed'].head().values

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  feats = literal_eval(x.replace('\\"','"'))
  for item in feats:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
    
  return output_dict  


df['features_parsed'] = df['features'].map(parse_features)


In [0]:
df['features_parsed'].head().values

In [30]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [31]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [36]:
key_stats = {}
for key in tqdm_notebook(keys):
  key_stats[get_name_feat(key)] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100


In [40]:
{k:v for k,v in key_stats.items() if v>30}

{'feat_brand': 48.62691466083151,
 'feat_color': 47.784463894967175,
 'feat_gender': 50.17505470459519,
 'feat_manufacturer part number': 36.252735229759296,
 'feat_material': 34.9070021881838}

In [68]:
df['feat_brand_cat'] = df.feat_brand.factorize()[0]
df['feat_color_cat'] = df.feat_color.factorize()[0]
df['feat_gender_cat'] = df.feat_gender.factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df.feat_material.factorize()[0]

df['feat_sport_cat'] = df.feat_sport.factorize()[0]
df['feat_style_cat'] = df.feat_style.factorize()[0]

feats_cat = [];
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  feats_cat.append(get_name_feat(key) + '_cat')

feats_cat

['feat_productgroup_cat',
 'feat_waterproof_cat',
 'feat_to fit_cat',
 'feat_autographed_cat',
 'feat_number of pieces_cat',
 'feat_wheel type_cat',
 'feat_polarized?_cat',
 'feat_removable liner_cat',
 'feat_fits models_cat',
 'feat_special features_cat',
 'feat_licensed_cat',
 'feat_closure_cat',
 'feat_antiscratch lens coating_cat',
 'feat_model number_cat',
 'feat_number of wheels_cat',
 'feat_style_cat',
 'feat_pocket_cat',
 'feat_display technology_cat',
 'feat_height_cat',
 'feat_dial material_cat',
 'feat_expandable_cat',
 'feat_net weight_cat',
 'feat_frame material_cat',
 'feat_target audience_cat',
 'feat_guaranteed authentic_cat',
 'feat_adjustable_cat',
 'feat_fits most screen size_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_location - country_cat',
 'feat_smart watch_cat',
 'feat_labor_cat',
 'feat_item style_cat',
 'feat_age gender group_cat',
 'feat_cm_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_jewelry setting_cat',
 'feat_inv_cat',
 'feat_case fin

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [42]:
df[ df.brand == df.feat_brand ].shape

(8846, 527)

In [107]:
#feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_size_cat','feat_condition_cat','feat_assembled product dimensions (l x w x h)_cat',
'feat_weight_cat','feat_casual & dress shoe style_cat','feat_country of origin - components_cat'
]
#model = RandomForestRegressor(n_estimators=100,max_depth=5, random_state=0)
model = RandomForestRegressor(n_estimators=100,max_depth=5)
result = run_model(feats, model)
print(result)

(-56.94820179958286, 4.204689469237294)


In [0]:
partial = []
for i in range(301,400):
  partial.append(feats_cat[i]);
feats = partial

In [108]:
x = df[ feats ].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(n_estimators=100,max_depth=5)
m.fit(x,y)
perm = PermutationImportance(m, random_state=1).fit(x,y)

print(result)
eli5.show_weights(perm, feature_names=feats)

(-56.94820179958286, 4.204689469237294)


Weight,Feature
0.2579 ± 0.0116,brand_cat
0.1053 ± 0.0029,feat_material_cat
0.0442 ± 0.0020,feat_gender_cat
0.0190 ± 0.0016,feat_brand_cat
0.0142 ± 0.0016,feat_weight_cat
0.0068 ± 0.0009,feat_style_cat
0.0024 ± 0.0011,feat_condition_cat
0.0012 ± 0.0002,feat_casual & dress shoe style_cat
0.0011 ± 0.0003,feat_size_cat
0.0004 ± 0.0000,feat_country of origin - components_cat
